In [1]:
from spx_predictor import SPXPredictor
from spx_feature_explorer import SPXFeatureExplorer

# Train baseline
predictor = SPXPredictor()
predictor.train(years=7)

# The predictor already has the data, let me re-fetch it
from UnifiedDataFetcher import UnifiedDataFetcher
from datetime import datetime, timedelta

fetcher = UnifiedDataFetcher()
end_date = datetime.now()
start_date = end_date - timedelta(days=7 * 365)
spx_df = fetcher.fetch_spx(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
spx = spx_df['Close'].squeeze()
spx.index = spx.index.tz_localize(None)

# Explore
explorer = SPXFeatureExplorer()
results = explorer.run_full_analysis(
    base_features=predictor.features_scaled,
    spx=spx,
    target='direction_21d'
)


SPX PREDICTION SYSTEM

📊 Fetching data: 2018-10-24 to 2025-10-22
✓ Using cached: yahoo_^GSPC_2018-10-24_2025-10-22.parquet
✓ Using cached: vix_2018-10-24_2025-10-22.parquet
✓ Using cached: macro_2018-10-24_2025-10-22.parquet
✓ Using cached: fred_all_2018-10-24_2025-10-22.parquet
✅ Using ONLY backward-looking volatility features (no cheating)
✅ Data loaded

🔧 Building features...
✅ Features built: 1472 samples, 44 features


🔍 Selecting top 30 features...
✅ Selected 30 features

📊 TOP 30 FEATURES BY IMPORTANCE:
    1. 10Y-2Y Yield Spread_change_63                      0.0779
    2. spx_realized_vol_63                                0.0730
    3. 10Y Breakeven Inflation_level                      0.0690
    4. yield_slope                                        0.0504
    5. 10Y-2Y Yield Spread_level                          0.0415
    6. spx_vs_ma200                                       0.0359
    7. month                                              0.0352
    8. spx_realized_vol_21  

In [1]:
from spx_predictor import SPXPredictor

# Initialize
predictor = SPXPredictor()

# Train on 7 years of data
summary = predictor.train(years=7)
print(summary)

# Get current predictions
predictions = predictor.predict_current()

# View top features
top_features = predictor.get_feature_importance(top_n=10)
for feat, imp in top_features.items():
    print(f"{feat}: {imp:.4f}")


SPX PREDICTION SYSTEM - DATA COLLECTION

📊 Fetching data: 2018-10-25 to 2025-10-23
✓ Using cached: yahoo_^GSPC_2018-10-25_2025-10-23.parquet
✓ Using cached: vix_2018-10-25_2025-10-23.parquet
✓ Using cached: macro_2018-10-25_2025-10-23.parquet
✓ Using cached: fred_all_2018-10-25_2025-10-23.parquet
✅ Data loaded: 1757 trading days

🔧 Building features...
✅ Features built: 1472 samples, 44 features


🔍 Selecting top 30 features...
✅ Selected 30 features

📊 TOP 30 FEATURES BY IMPORTANCE:
    1. 10Y-2Y Yield Spread_change_63                      0.0823
    2. 10Y Breakeven Inflation_level                      0.0724
    3. spx_realized_vol_63                                0.0644
    4. yield_slope                                        0.0577
    5. 10Y-2Y Yield Spread_level                          0.0458
    6. spx_vs_ma200                                       0.0365
    7. month                                              0.0329
    8. spx_realized_vol_21                             

In [4]:
import yfinance as yf

spx = yf.Ticker("IAK")
price = spx.fast_info["last_price"]
print("price: ", price)



current_spx_realtime = yf.Ticker('^GSPC')
if current_spx_realtime is None:
    print("⚠️  Could not fetch real-time price, using model price")
    current_spx_realtime = current_spx_model
        
    current_vix = fetcher.fetch_price('^VIX')

price:  128.0399932861328


In [1]:
from spx_feature_explorer import SPXFeatureExplorer

explorer = SPXFeatureExplorer()
results = explorer.run_complete_analysis(years=7)


🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
COMPLETE FEATURE ANALYSIS PIPELINE
🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯

FETCHING DATA

📊 Date Range: 2018-10-24 to 2025-10-22
   Fetching SPX...
✓ Using cached: yahoo_^GSPC_2018-10-24_2025-10-22.parquet
   Fetching VIX...
✓ Using cached: vix_2018-10-24_2025-10-22.parquet

✅ SPX: 1757 days
✅ VIX: 1757 days

TRAINING BASELINE MODEL

SPX PREDICTION SYSTEM

📊 Fetching data: 2018-10-24 to 2025-10-22
✓ Using cached: yahoo_^GSPC_2018-10-24_2025-10-22.parquet
✓ Using cached: vix_2018-10-24_2025-10-22.parquet
✓ Using cached: macro_2018-10-24_2025-10-22.parquet
✓ Using cached: fred_all_2018-10-24_2025-10-22.parquet
✅ Using ONLY backward-looking volatility features (no cheating)
✅ Data loaded

🔧 Building features...
✅ Features built: 1472 samples, 44 features


🔍 Selecting top 30 features...
✅ Selected 30 features

📊 TOP 30 FEATURES BY IMPORTANCE:
    1. 10Y-2Y Yield Spread_change_63                      0.0779
    2. spx_realized_vol_63                       